In [1]:
# import model as M
# import evaluate as E
import config as CFG
import os
import json
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel, BatchEncoding, BertModel
from tqdm import tqdm

In [2]:
model_checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [3]:
DATASET_ROOT= 'exp/data/'
RESULT_ROOT = "exp/result_bert_base_uncased_summarly"
METHOD = 'summar_ly'

DATASETS = ["cnn_dailymail"]

In [4]:
def convert_single_example(example):
    doc_sents = ["[CLS] " + sent for sent in example["doc"]]
    doc_text = " ".join(doc_sents)

    samples = []

    for _sum in example["sums"]:
        sum_sents = ["[CLS] " + sent["text"] for sent in _sum["sample"]]
        labels = [[sent["fact"], sent["ling"]] for sent in _sum["sample"]]
        sum_text = " ".join(sum_sents)

        inputs = tokenizer(doc_text, sum_text, 
        padding='max_length', truncation="longest_first", return_tensors="pt")
        cov_label = [-100] * tokenizer.model_max_length
        sum_label = [[-100, -100]] * tokenizer.model_max_length
        
        # Align coverage label
        cnt = 0
        sep = -1
        for i in range(1, tokenizer.model_max_length):
            if inputs["input_ids"][0][i] == tokenizer.sep_token_id:
                sep = i
                break
            elif inputs["input_ids"][0][i] == tokenizer.cls_token_id:
                cov_label[i] = int(_sum["coverage"][cnt])
                cnt += 1
        cnt = 0
        # Align fact/ling label
        for i in range(sep, tokenizer.model_max_length):
            if inputs["input_ids"][0][i] == tokenizer.cls_token_id:
                sum_label[i] = labels[cnt]
                cnt += 1

        final_label = np.hstack([np.array(cov_label).reshape(-1, 1), np.array(sum_label)])
        
        inputs = {k: v.squeeze() for k, v in inputs.items()}
        samples.append((inputs, torch.Tensor(final_label)))
        
    return samples

In [5]:
class TokenizedDataset(Dataset):
    def __init__(self, datapath, limit = None):
        self.data = []
        with open(datapath, "r", encoding="utf-8") as f:
            for line in tqdm(f):
                example = json.loads(line)
                samples = convert_single_example(example)
                size = len(samples)
                for i in range(1, size):
                    self.data.append((samples[0], samples[i]))
                
                if not limit is None:
                    limit -= 1
                    if limit == 0: break

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx][0], self.data[idx][1]

In [6]:
class TokenizedTestset(Dataset):
    def __init__(self, datapath, limit = None):
        self.data = []
        with open(datapath, "r", encoding="utf-8") as f:
            for line in tqdm(f):
                example = json.loads(line)
                samples = convert_single_example(example)
                self.data.extend(samples)

                if not limit is None:
                    limit -= 1
                    if limit == 0: break

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

In [5]:
class Scorer(nn.Module):
    def __init__(self):
        super(Scorer, self).__init__()
        self.model = AutoModel.from_pretrained(CFG.BERT_MODEL)
        self.score_head = nn.Linear(self.model.config.hidden_size, 1)
        self.classify_head = nn.Linear(self.model.config.hidden_size, 3)
    
    def forward(self, inputs):
        outputs = self.model(**inputs)
        score = self.score_head(outputs.pooler_output)
        logits = self.classify_head(outputs.last_hidden_state)

        return score, logits

In [8]:
def test(model, test_set):
    model.eval()
    test_dataloader = DataLoader(test_set, batch_size=CFG.BATCH_SIZE)
    tps = 0
    fps = 0
    fns = 0
    with torch.no_grad():
        for batch in test_dataloader:
            inputs, labels = batch
            labels = labels.to(CFG.DEVICE)
            inputs = BatchEncoding(inputs).to(CFG.DEVICE)
            
            score, logits = model(inputs)
            preds = F.sigmoid(logits) >= 0.5
            tp = torch.sum((preds == 1) * (labels == 1))
            fp = torch.sum((preds == 1) * (labels == 0))
            fn = torch.sum((preds == 0) * (labels == 1))
            tps += tp.item()
            fps += fp.item()
            fns += fn.item()

    model.train()
    precision = tps / (fps + tps)
    recall = tps / (tps + fns)
    return precision, recall


In [9]:
def train_model(model, train_set, test_set = None, max_iter=CFG.MAX_ITERATION):
    train_dataloader = DataLoader(train_set, batch_size=CFG.BATCH_SIZE, shuffle=True)
    optimizer = torch.optim.Adam(model.parameters(), lr = CFG.LR)
    score_loss_fn = nn.CrossEntropyLoss()
    class_loss_fn = nn.BCEWithLogitsLoss(reduction='none')

    running_loss = 0.0
    num_iter = 0
    with tqdm(total=max_iter) as pbar:
        while num_iter < max_iter:
            for pos, neg in train_dataloader:
                if num_iter >= max_iter:
                    break
                inputs_pos, labels_pos = pos
                inputs_neg, labels_neg = neg
                labels_pos = labels_pos.to(CFG.DEVICE)
                labels_neg = labels_neg.to(CFG.DEVICE)
                inputs_pos = BatchEncoding(inputs_pos).to(CFG.DEVICE)
                inputs_neg = BatchEncoding(inputs_neg).to(CFG.DEVICE)

                score_pos, logits_pos = model(inputs_pos)
                score_neg, logits_neg = model(inputs_neg)
                mask_pos = labels_pos != -100
                mask_neg = labels_neg != -100

                # Preference Loss
                score_cat = torch.cat((score_pos, score_neg), -1)
                score_labels = torch.tensor([0]*labels_pos.shape[0], dtype=torch.long).to(CFG.DEVICE)
                loss_score = score_loss_fn(score_cat, score_labels)
                # Classification Loss
                loss_class_pos = class_loss_fn(logits_pos, labels_pos) * mask_pos
                loss_class_neg = class_loss_fn(logits_neg, labels_neg) * mask_neg
                loss = loss_score + torch.sum(loss_class_pos)/torch.sum(mask_pos) + torch.sum(loss_class_neg) / torch.sum(mask_neg)

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                running_loss += loss.item()

                num_iter += 1
                pbar.update(1)
                if num_iter % 1000 == 999:
                    pbar.write("Iteration {}, Loss {}".format(num_iter+1, running_loss))
                    running_loss = 0.0
                    if not test_set is None:
                        pbar.write("Testing...")
                        p, r = test(model, test_set)
                        pbar.write("Precision {:.2f}, recall {:.2f}".format(p, r))
                        

In [10]:
def train():
    for DATASET in DATASETS:
        train_set = TokenizedDataset(os.path.join(DATASET_ROOT, DATASET, METHOD, 'train.jsonl'))
        test_set = TokenizedTestset(os.path.join(DATASET_ROOT, DATASET, METHOD, 'test.jsonl'))
        
        model = Scorer()
        model.to(CFG.DEVICE)

        train_model(model, train_set, test_set)

        CKPT_PATH = os.path.join(RESULT_ROOT, DATASET, METHOD, "model.pth")
        if not os.path.exists(os.path.dirname(CKPT_PATH)):
            os.makedirs(os.path.dirname(CKPT_PATH))

        torch.save(model.state_dict(), CKPT_PATH)

        del model
        torch.cuda.empty_cache()

In [11]:
train()

40000it [1:05:45, 10.14it/s]
11490it [19:40,  9.74it/s]
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
  1%|█▊                                                                                            

Iteration 1000, Loss 960.823295712471
Testing...


  1%|█▊                                                                                                                                                                                      | 999/100000 [09:14<7:46:42,  3.54it/s]

Precision 0.86, recall 0.51


  2%|███▋                                                                                                                                                                                   | 1999/100000 [14:13<8:19:44,  3.27it/s]

...

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉| 99999/100000 [15:44:25<00:00,  3.33it/s]

Iteration 100000, Loss 158.74481904879212
Testing...


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉| 99999/100000 [15:48:56<00:00,  3.33it/s]

Precision 0.85, recall 0.79


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100000/100000 [15:48:57<00:00,  1.76it/s]


In [6]:
DATASET = DATASETS[0]
# test_set = TokenizedTestset(os.path.join(DATASET_ROOT, DATASET, METHOD, 'test.jsonl'), 10)
CKPT_PATH = os.path.join(RESULT_ROOT, DATASET, METHOD, "model.pth")
model = Scorer()
model.load_state_dict(torch.load(CKPT_PATH, map_location=CFG.DEVICE))
model.to(CFG.DEVICE)
# test(model, test_set)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Scorer(
  (model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    

In [7]:
from utils import sent_tokenizer

ImportError: cannot import name 'get_terminal_size' from 'click.termui' (/home/gluo/miniconda3/lib/python3.8/site-packages/click/termui.py)

In [15]:
def evaluate(docs, sums, model, score_output, detail_output):
    docs = sent_tokenizer(docs)
    sums = sent_tokenizer(sums)

    doc_texts, sum_texts = []
    for _doc, _sum in zip(docs, sums):
        doc_sents = ["[CLS] " + sent for sent in _doc]
        sum_sents = ["[CLS] " + sent for sent in _sum]
        doc_texts.append(" ".join(doc_sents))
        sum_texts.append(" ".join(sum_texts))

    inputs = tokenizer(doc_texts, sum_texts, 
        padding='max_length', truncation="longest_first", return_tensors="pt") 
    
    eval_loader = DataLoader(inputs, batch_size=CFG.BATCH_SIZE)
    
    scores = []
    preds = []
    with torch.no_grad():
        for batch in eval_loader:
            batch = batch.to(CFG.DEVICE)
            
            score, logits = model(inputs)
            scores.append(score.squeeze().detach().cpu().numpy())
            preds.append(preds.detach().cpu().numpy())
    
    scores = np.vstack(scores)
    preds = np.vstack(preds)
    
    with open(score_output, "w", encoding = "UTF-8") as f:
        f.write("\n".join([str(score) for score in scores])+"\n")
    